In [1]:
# ===== Colab Cell 1: Library (real-only Hermite/CH with symbolic n, t) =====
# !pip -q install sympy

import sympy as sp
x = sp.Symbol('x')
I = sp.I

# ---------- helpers ----------
def vec_col_major(M):
    M = sp.Matrix(M)
    n, m = M.shape
    return sp.Matrix(n*m, 1, lambda i, _: M[i % n, i // n])

def evaluate_on_matrix(A, coeffs):
    """
    v(A) for v(x)=sum_{k=0}^{q-1} a_k x^k, with symbolic coefficients a_k(n) or a_k(t).
    """
    A = sp.Matrix(A)
    m = A.shape[0]
    res = sp.zeros(m)
    P = sp.eye(m)
    for k, ak in enumerate(coeffs):
        if k == 0:
            res += ak * sp.eye(m)
        else:
            P = P * A
            res += ak * P
    return sp.simplify(res)

# ---------- minimal polynomial via linear system B_A ----------
def minimal_polynomial_via_linear(A):
    A = sp.Matrix(A)
    n = A.shape[0]
    for q in range(1, n+1):
        B_cols = [vec_col_major(A**k) for k in range(q)]
        B = sp.Matrix.hstack(*B_cols)
        b = vec_col_major(A**q)
        sol = sp.linsolve((B, -b))
        if sol != sp.EmptySet:
            c = list(next(iter(sol)))
            poly = x**q + sum(c[k] * x**k for k in range(q))
            return sp.poly(sp.factor(poly), x).monic()
    # fallback
    try:
        expr = A.minimal_polynomial(x)
        return sp.poly(expr, x).monic()
    except Exception:
        return A.charpoly(x).as_poly().monic()


def factor_minpoly(mA):
    """
    Factor m_A over C → (roots, mults) with correct multiplicities.
    """
    expr = sp.factor(mA.as_expr())
    _, plist = sp.factor_list(expr)
    roots, mults = [], []
    for p, e in plist:
        inner = sp.roots(p)
        for lam, mr in inner.items():
            roots.append(sp.nsimplify(lam))
            mults.append(e * mr)
    return roots, mults

# ---------- split roots to real & conjugate pairs ----------
def split_real_conj(roots, mults):
    used = [False]*len(roots)
    reals, pairs, fallback = [], [], False
    for i, (ri, mi) in enumerate(zip(roots, mults)):
        if used[i]:
            continue
        if sp.simplify(sp.im(ri)) == 0:
            reals.append((sp.nsimplify(ri), mi))
            used[i] = True
        else:
            ci = sp.conjugate(ri)
            j_found = None
            for j in range(i+1, len(roots)):
                if not used[j] and sp.simplify(roots[j] - ci) == 0 and mults[j] == mi:
                    j_found = j
                    break
            if j_found is None:
                fallback = True
                used[i] = True
            else:
                # keep representative with Im>0
                if sp.im(ri) < 0:
                    ri = ci
                pairs.append((sp.nsimplify(ri), mi))
                used[i] = True
                used[j_found] = True
    return reals, pairs, fallback

# ---------- build REAL confluent-Vandermonde (polar for complex roots) ----------
def real_hermite_system(A, roots, mults, mode, n=None, t=None):
    """
    Build real system M a = y for v(x)=sum_{k=0}^{q-1} a_k x^k.
    mode ∈ {'power','exp','inv'}, with symbolic n or t allowed.
    """
    reals, pairs, fallback = split_real_conj(roots, mults)
    q = sum(mults)
    rows, rhs = [], []

    fact = sp.factorial
    ff   = sp.FallingFactorial # Corrected from sp.fallingfactorial

    # real roots
    for lam, r in reals:
        for j in range(r):
            row = [ (fact(k)//fact(k-j) * lam**(k-j) if k>=j else 0) for k in range(q) ]
            rows.append(row)
            if mode == 'power':
                rhs.append(sp.simplify(ff(n, j) * lam**(n - j)))
            elif mode == 'exp':
                rhs.append(sp.simplify((t**j) * sp.exp(t*lam)))
            elif mode == 'inv':
                rhs.append(0 if sp.simplify(lam)==0 else ((-1)**j)*fact(j)/lam**(j+1))
            else:
                raise ValueError("mode?")

    # complex conjugate pairs → two real rows (cos/sin), polar form
    for z, rmult in pairs:
        rho, theta = sp.Abs(z), sp.arg(z)
        for j in range(rmult):
            rowR, rowI = [], []
            for k in range(q):
                if k < j:
                    rowR.append(0); rowI.append(0)
                else:
                    m = k - j
                    c = fact(k)//fact(k-j)
                    rowR.append(c * rho**m * sp.cos(m*theta))
                    rowI.append(c * rho**m * sp.sin(m*theta))
            rows += [rowR, rowI]
            if mode == 'power':
                p  = n - j
                coef = ff(n, j)
                rhs += [ sp.simplify(coef * rho**p * sp.cos(p*theta)),
                         sp.simplify(coef * rho**p * sp.sin(p*theta)) ]
            elif mode == 'exp':
                rhs += [ sp.simplify((t**j)*sp.exp(t*rho*sp.cos(theta))*sp.cos(t*rho*sp.sin(theta))),
                         sp.simplify((t**j)*sp.exp(t*rho*sp.cos(theta))*sp.sin(t*rho*sp.sin(theta))) ]
            elif mode == 'inv':
                p   = -(j+1)
                coef = ((-1)**j)*fact(j)
                rhs += [ sp.simplify(coef * rho**p * sp.cos(p*theta)),
                         sp.simplify(coef * rho**p * sp.sin(p*theta)) ]
            else:
                raise ValueError("mode?")

    M = sp.Matrix(rows)
    y = sp.Matrix(rhs)
    assert M.shape == (q, q), f"bad size {M.shape} vs ({q},{q})"
    return M, y, fallback

def matfun_real_hermite(A, mode, n=None, t=None):
    """
    Compute f(A) with f(x)=x^n (symbolic n), f(x)=e^{tx} (symbolic t), or f(x)=1/x (Drazin kernel),
    via minimal polynomial + REAL Hermite system (polar for complex roots).
    """
    A = sp.Matrix(A)
    mA = minimal_polynomial_via_linear(A)
    roots, mults = factor_minpoly(mA)
    M, y, _ = real_hermite_system(A, roots, mults, mode, n=n, t=t)
    coeffs = list(M.LUsolve(y))         # symbolic dependence appears here
    return evaluate_on_matrix(A, coeffs)

# ---------- public wrappers ----------
def power_via_CH_symbolic(A, n_symbol=None):
    """
    Return A^n symbolically (n stays as a symbol).
    """
    n = n_symbol or sp.Symbol('n', integer=True)
    return sp.simplify(matfun_real_hermite(A, 'power', n=n))

def exp_via_CH_symbolic(A, t_symbol=None):
    """
    Return exp(t A) symbolically (t stays as a symbol).
    """
    t = t_symbol or sp.Symbol('t', real=True)
    return sp.simplify(matfun_real_hermite(A, 'exp', t=t))

def drazin_via_hermite(A):
    """
    Drazin inverse via f(x)=1/x for x≠0, f(0)=0 in the Hermite system (keeps everything real).
    """
    return sp.simplify(matfun_real_hermite(A, 'inv'))

In [2]:
# ===== Colab Cell 2: Symbolic demos (n, t remain as symbols) =====
import sympy as sp
from IPython.display import display, Math

# Init pretty printing with LaTeX/MathJax
sp.init_printing(use_latex='mathjax')

# Define symbols
n = sp.Symbol('n', integer=True)
t = sp.Symbol('t', real=True)

# Example 1: 2x2 with complex spectrum → A^n symbolic
A = sp.Matrix([[3, 2],
               [-1, 3]])
A_n = sp.simplify(power_via_CH_symbolic(A, n))
display(Math(r"A^n = " + sp.latex(A_n)))

# Example 2: exp(tA) symbolic
EtA = sp.simplify(exp_via_CH_symbolic(A, t))
display(Math(r"\exp(tA) = " + sp.latex(EtA)))

# Example 3: a 3x3 with mixed eigenvalues (1 double, and a complex pair if present)
P = sp.Matrix([[0, 1, 0],
               [0, sp.Rational(2,3), sp.Rational(1,3)],
               [sp.Rational(1,3), 0, sp.Rational(2,3)]])
P_n = sp.simplify(power_via_CH_symbolic(P, n))
display(Math(r"P^n = " + sp.latex(P_n)))

E_tP = sp.simplify(exp_via_CH_symbolic(P, t))
display(Math(r"\exp(tP) = " + sp.latex(E_tP)))

# Optional: Drazin inverse of a singular matrix, stays exact/rational
C = sp.Matrix([[sp.Rational(1,5),  sp.Rational(2,5),  sp.Rational(2,5)],
               [sp.Rational(3,10), sp.Rational(3,5),  sp.Rational(1,10)],
               [sp.Rational(1,10), sp.Rational(1,5),  sp.Rational(7,10)]])
C_D = sp.simplify(drazin_via_hermite(C))
display(Math(r"C^D = " + sp.latex(C_D)))

# Quick sanity check: plug a concrete n to see it collapses correctly
A5 = sp.simplify(A_n.subs(n, 5))
display(Math(r"A^5 \; (\text{check with } n=5) = " + sp.latex(A5)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
# ===== Verification helpers =====
import sympy as sp

def _is_zero_matrix(M):
    """Entrywise simplify-to-zero check (robust for symbolic matrices)."""
    M = sp.Matrix(M)
    return all(sp.simplify(M[i, j]) == 0 for i in range(M.rows) for j in range(M.cols))

def isan(An, A, n_symbol=None):
    """
    Verify A^n by comparing (A^n)A with A^{n+1} produced by the same CH/Hermite routine.
    Returns True if they match symbolically (after simplify), else False.
    """
    n = n_symbol or sp.Symbol('n', integer=True)
    lhs = sp.simplify(sp.Matrix(An) * sp.Matrix(A))
    rhs = sp.simplify(power_via_CH_symbolic(A, n + 1))
    return _is_zero_matrix(lhs - rhs)

def isexpt(EtA, A, t_symbol=None):
    """
    Verify exp(tA) via the IVP:
      (d/dt)E(t) = A E(t)  and  E(0) = I.
    Returns True if both conditions hold symbolically, else False.
    """
    t = t_symbol or sp.Symbol('t', real=True)
    A = sp.Matrix(A)
    EtA = sp.Matrix(EtA)

    cond_diff = _is_zero_matrix(sp.simplify(sp.diff(EtA, t) - A * EtA))
    cond_init = _is_zero_matrix(sp.simplify(EtA.subs(t, 0) - sp.eye(A.shape[0])))
    return cond_diff and cond_init

def _drazin_index(A):
    """
    Compute the Drazin index κ = min{k >= 0 : rank(A^{k+1}) = rank(A^k)}.
    For invertible A, κ = 0.
    """
    A = sp.Matrix(A)
    n = A.shape[0]
    # Quick exit if invertible
    try:
        if A.det() != 0:
            return 0
    except Exception:
        pass  # det may be symbolic; fall back to rank stabilization

    for k in range(n):  # k = 0,1,...,n-1 suffices
        rk  = (A**k).rank()
        rk1 = (A**(k+1)).rank()
        if rk1 == rk:
            return k
    return n - 1  # very conservative fallback

def isdrazin(X, A, return_details=False):
    """
    Verify that X is the Drazin inverse of A by checking:
      1) A^{κ+1} X = A^{κ}
      2) AX = XA
      3) XAX = X
    where κ is the Drazin index of A.

    Returns:
      - bool  (or (bool, details_dict) if return_details=True)
    """
    A = sp.Matrix(A)
    X = sp.Matrix(X)
    kappa = _drazin_index(A)

    c1 = _is_zero_matrix(sp.simplify(A**(kappa + 1) * X - A**kappa))
    c2 = _is_zero_matrix(sp.simplify(A * X - X * A))
    c3 = _is_zero_matrix(sp.simplify(X * A * X - X))

    ok = c1 and c2 and c3
    if return_details:
        return ok, {"kappa": kappa, "A^(k+1)X=A^k": c1, "AX=XA": c2, "XAX=X": c3}
    return ok


In [7]:
# Example matrix from your notes
A = sp.Matrix([[3,2],[-1,3]])
n = sp.Symbol('n', integer=True)
t = sp.Symbol('t', real=True)

An = power_via_CH_symbolic(A, n)
EtA = exp_via_CH_symbolic(A, t)

print("isan:", isan(An, A, n))
print("isexpt:", isexpt(EtA, A, t))

# Drazin example
C = sp.Matrix([[sp.Rational(1,5),  sp.Rational(2,5),  sp.Rational(2,5)],
               [sp.Rational(3,10), sp.Rational(3,5),  sp.Rational(1,10)],
               [sp.Rational(1,10), sp.Rational(1,5),  sp.Rational(7,10)]])
CD = drazin_via_hermite(C)
print("isdrazin:", isdrazin(CD, C, return_details=True))


isan: True
isexpt: True
isdrazin: (True, {'kappa': 1, 'A^(k+1)X=A^k': True, 'AX=XA': True, 'XAX=X': True})
